# Quantum vs Classical Model Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/02_quantum_vs_classical.ipynb)

This tutorial demonstrates how to train and compare quantum-enhanced and classical models for protein folding.

## Topics Covered
1. Installation and setup
2. Data preparation
3. Training quantum models
4. Training classical baselines
5. Performance benchmarking
6. Statistical comparison

## Step 1: Installation

Clone repository and install dependencies.

In [ ]:
# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally")

# Check GPU
import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
if IN_COLAB:
    print("Installing QuantumFold-Advantage...")
    !git clone https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git
    %cd QuantumFold-Advantage
    
    print("\nFixing JAX version for PennyLane compatibility...")
    !pip install -q 'jax==0.6.0' 'jaxlib==0.6.0'
    
    print("\nInstalling dependencies...")
    !pip install -q pennylane matplotlib pandas scikit-learn biopython requests tqdm
    
    print("Installation complete!")

## Step 2: Import Libraries

In [ ]:
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
from pathlib import Path

# Add src to path
if IN_COLAB:
    sys.path.insert(0, '/content/QuantumFold-Advantage')
else:
    sys.path.insert(0, str(Path.cwd().parent))

from src.quantum_layers import QuantumAttentionLayer
from src.benchmarks import ProteinStructureEvaluator

print("Imports successful!")

## Step 3: Prepare Training Data

In [ ]:
# Generate synthetic training data
np.random.seed(42)
torch.manual_seed(42)

n_samples = 100
seq_length = 50
feature_dim = 64

X_train = torch.randn(n_samples, seq_length, feature_dim)
y_train = torch.randn(n_samples, seq_length, 3)

X_test = torch.randn(20, seq_length, feature_dim)
y_test = torch.randn(20, seq_length, 3)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

## Step 4: Define Models

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class QuantumModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.quantum = QuantumAttentionLayer(
            embed_dim=feature_dim,
            n_qubits=4,
            n_heads=4
        )
        self.output = nn.Linear(feature_dim, 3)
    
    def forward(self, x):
        x = self.quantum(x)
        return self.output(x)

class ClassicalModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention = nn.MultiheadAttention(feature_dim, 4, batch_first=True)
        self.output = nn.Linear(feature_dim, 3)
    
    def forward(self, x):
        x, _ = self.attention(x, x, x)
        return self.output(x)

print("Initializing models...")
quantum_model = QuantumModel().to(device)
classical_model = ClassicalModel().to(device)

print(f"\nModels initialized on {device}")
print(f"Quantum parameters:   {sum(p.numel() for p in quantum_model.parameters()):,}")
print(f"Classical parameters: {sum(p.numel() for p in classical_model.parameters()):,}")

## Step 5: Train Models

In [ ]:
def train_model(model, X, y, epochs=10, lr=0.001, model_name='Model'):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    losses = []
    times = []
    X, y = X.to(device), y.to(device)
    
    print(f"\nTraining {model_name}...")
    for epoch in range(epochs):
        start_time = time.time()
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        epoch_time = time.time() - start_time
        losses.append(loss.item())
        times.append(epoch_time)
        
        if (epoch + 1) % 2 == 0:
            print(f"  Epoch {epoch+1:2d}/{epochs} | Loss: {loss.item():.4f} | Time: {epoch_time:.2f}s")
    
    print(f"{model_name} training complete!")
    return losses, times

print("\nQUANTUM MODEL")
print("=" * 60)
q_losses, q_times = train_model(quantum_model, X_train, y_train, epochs=10, model_name='Quantum')

print("\nCLASSICAL MODEL")
print("=" * 60)
c_losses, c_times = train_model(classical_model, X_train, y_train, epochs=10, model_name='Classical')

## Step 6: Compare Performance

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(q_losses, 'b-', label='Quantum', linewidth=2, marker='o', markersize=4)
ax1.plot(c_losses, 'r-', label='Classical', linewidth=2, marker='s', markersize=4)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss (MSE)', fontsize=12)
ax1.set_title('Training Loss Comparison', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

ax2.plot(np.cumsum(q_times), 'b-', label='Quantum', linewidth=2, marker='o', markersize=4)
ax2.plot(np.cumsum(c_times), 'r-', label='Classical', linewidth=2, marker='s', markersize=4)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Cumulative Time (s)', fontsize=12)
ax2.set_title('Training Time Comparison', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPERFORMANCE SUMMARY")
print("=" * 60)
print(f"\nQuantum Model:")
print(f"  Final Loss:  {q_losses[-1]:.6f}")
print(f"  Total Time:  {sum(q_times):.2f}s")
print(f"  Avg per Epoch: {np.mean(q_times):.2f}s")

print(f"\nClassical Model:")
print(f"  Final Loss:  {c_losses[-1]:.6f}")
print(f"  Total Time:  {sum(c_times):.2f}s")
print(f"  Avg per Epoch: {np.mean(c_times):.2f}s")

speedup = sum(c_times) / sum(q_times)
print(f"\nSpeed Comparison:")
if speedup > 1:
    print(f"  Classical is {speedup:.2f}x faster")
else:
    print(f"  Quantum is {1/speedup:.2f}x faster")

loss_improvement = (c_losses[-1] - q_losses[-1]) / c_losses[-1] * 100
print(f"\nLoss Comparison:")
if loss_improvement > 0:
    print(f"  Quantum achieves {loss_improvement:.1f}% lower loss")
else:
    print(f"  Classical achieves {-loss_improvement:.1f}% lower loss")

print("\n" + "=" * 60)

## Summary

In this tutorial, we:

1. Set up training pipeline
2. Trained quantum model with variational circuits
3. Trained classical baseline with attention
4. Compared training dynamics
5. Analyzed performance metrics

### Key Observations

- Quantum models may show different convergence patterns
- Training time depends on circuit depth and qubit count
- Both approaches can achieve competitive performance
- The quantum advantage may be more pronounced on real hardware

### Next Steps

Explore more:
- [Advanced Visualization](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/03_advanced_visualization.ipynb)
- [Getting Started](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/01_getting_started.ipynb)